In [1]:
import momi
import pandas
import os

Use the **`help()`** function to view documentation.

In [2]:
help(momi)

Help on package momi:

NAME
    momi

FILE
    /home/snackattack/anaconda/lib/python2.7/site-packages/momi/__init__.py

DESCRIPTION
    momi (MOran Models for Inference) is a python package for computing the site frequency spectrum,
    a summary statistic commonly used in population genetics, and using it to infer demographic history.
    
    Please refer to examples/tutorial.ipynb for usage & introduction.

PACKAGE CONTENTS
    compute_sfs
    convolution
    data_structure
    demography
    likelihood
    math_functions
    moran_model
    parse_ms
    size_history
    tensor
    util




# Creating demographies

momi uses a syntax based on the program ms by Richard Hudson.
A demography is specified as a sequence of events.
Time is measured going backwards from the present (t=0) to the past (t>0).

There are 4 kinds of events:
* **('-en', t, i, N)**
    * At time t, population i has its scaled population size set to N, and growth rate set to 0.
* **('-eg', t, i, g)**
    * At time t, population i has exponential growth rate g (so for s>t, $N(s) = N(t) e^{(s-t)  g}$)
* **('-ej', t, i, j)**
    * At time t, all lineages in population i move into j.
* **('-ep', t, i, j, p_ij)**
    * At time t, each lineage in i moves into j with probability p_ij.

Note **-en,-eg,-ej** are flags from ms, while **-ep** replaces the flag **-es** in ms.
By default, all parameters are scaled as in ms, but this can be adjusted.

See **`help(momi.Demography)`** or **`help(momi.Demography.__init__)`** for more details.

### An example demography

Now let's consider a concrete example. More examples can be found at [example_demographies.ipynb](files/example_demographies.ipynb).

Unlike ms, populations can be labeled by arbitrary strings. In this example, we'll label the sampled populations as **'chb'** and **'yri'**. The demography will also involve admixture with a third population, **'nea'**.

Using the default parameter scaling (which is the same as **ms**),
we assume that all population sizes have been rescaled by a "reference" size N_ref (e.g. 10,000),
and time is scaled so there are 4*N_ref generations per unit time.

In [3]:
# define the list of events
events = [('-en', 0., 'chb', 10.),         # at present (t=0), 'chb' has diploid population size 10 * N_ref
          ('-eg', 0, 'chb' , 6.),          # at present (t=0), 'chb' growing at rate 6
          ('-ep', .25, 'chb', 'nea', .03), # at t=.25, 'chb' has a bit of admixture from 'nea'
          ('-ej', .5, 'chb', 'yri'),       # at t=.5, 'chb' joins onto 'yri' 
          ('-ej', 1.5, 'yri', 'nea'),      # at t=1.5, 'yri' joins onto 'nea'
          ]

# construct the Demography object, sampling 14 alleles from 'yri' and 10 alleles from 'chb'
demo = momi.Demography(events, sampled_pops=('yri','chb'), sampled_n=(14,10))

# Coalescent statistics

Let's examine some statistics of the above demography, such as the TMRCA (time to most recent common ancestor) and total branch length of the genealogy.

In [4]:
eTmrca = momi.expected_tmrca(demo)
print "Expected TMRCA of all samples:", "\t", eTmrca

eTmrca_chb = momi.expected_deme_tmrca(demo, 'chb')
print "Expected TMRCA of chb samples:", "\t", eTmrca_chb

eL = momi.expected_total_branch_len(demo)
print "Expected total branch length:", "\t", eL

# See help(momi.expected_tmrca), etc. for more details.
# Advanced users can use momi.expected_sfs_tensor_prod()
# to compute these and many other summary statistics.

Expected TMRCA of all samples: 	1.41920517653
Expected TMRCA of chb samples: 	1.25374295456
Expected total branch length: 	7.93963743415


# Expected Sample Frequency Spectrum (SFS)

The expected SFS for configuration $((a_0,d_0),(a_1,d_1),...)$ is the expected number of SNPs with $a_0$ ancestral and $d_0$ derived alleles in population 0, $a_1$ ancestral and $d_1$ derived alleles in population 1, etc.

In the below example we use **`momi.expected_sfs()`** to compute the expected SFS for several configurations.

In [5]:
# a list of configs (index0 == yri, index1 == chb)
config_list = [( (14,0), (9,1), ), # 1 derived allele in chb
               ( (13,1), (10,0), ), # 1 derived allele in yri 
               ( (11,3), (9,1),) , # 3 derived in yri, 1 derived in chb
               ( (14,0), (0,10), ), # 0 derived in yri, all derived in chb
               ( (2,12), (10,0), ), # 12 derived in yri, 0 derived in chb 
               ( (2,12), (2,8), ), # 12 derived in yri, 8 derived in chb
               ]

# the SFS entries corresponding to each config in config_list
eSFS = momi.expected_sfs(demo, config_list, mut_rate=1.0)
print eSFS

# See help(momi.expected_sfs) for more options (e.g. folded SFS, sampling error, normalization)

[ 2.6309565   0.95647102  0.0048574   0.06108049  0.04371188  0.00323204]


# Segregating sites

A dataset of segregating sites has been stored in [tutorial_data.txt](tutorial_data.txt). The file is organized as follows:
* Each locus starts with a line "**//**".
* Subsequent lines correspond to segregating sites.
    * The first column is the **position** of the site $x \in [0,1]$.
    * Subsequent columns give the **ancestral,derived** allele counts (respectively), in each population.

In [6]:
# print first few lines in the shell
!head tutorial_data.txt

Position	:	yri	chb

//

0.0	:	14,0	7,3
0.0007	:	13,1	10,0
0.001	:	0,14	10,0
0.0012	:	11,3	10,0
0.0012	:	14,0	9,1
0.0013	:	0,14	10,0


In [7]:
# read the file with momi
data_filename = "tutorial_data.txt"
with open(data_filename,'r') as f:
    seg_sites = momi.read_seg_sites(f)

`seg_sites` is a list, with each entry corresponding to a locus.  
The i-th entry `seg_sites[i]` is a list of the `(position,config)` of each site.  
So `seg_sites` looks something like this:  
> `[[(position`$_{00}$ `, config`$_{00}$ `), (position`$_{01}$ `, config`$_{01}$ `), ...],`  
> &nbsp;&nbsp;`[(position`$_{10}$ `, config`$_{10}$ `), (position`$_{11}$ `, config`$_{11}$ `), ...], ...]`

In addition, `seg_sites` has an attribute `seg_sites.sampled_pops` giving the populations.

In [8]:
print "Populations:", seg_sites.sampled_pops
print "Number of loci:", len(seg_sites)
print "Number of sites on 0th locus:", len(seg_sites[0])
print "First 3 sites on the 0th locus:"
print seg_sites[0][:3]

Populations: ('yri', 'chb')
Number of loci: 20
Number of sites on 0th locus: 3525
First 3 sites on the 0th locus:
[(0.0, ((14, 0), (7, 3))), (0.0007, ((13, 1), (10, 0))), (0.001, ((0, 14), (10, 0)))]


# Observed SFS

Now let's convert the list of segregating sites into SFS.  

In [9]:
# a list, with the SFS at each locus
sfs_list = momi.get_sfs_list(seg_sites)

momi represents the SFS at each locus as a **dictionary**, mapping configs (tuples) to counts (ints).  
So `sfs_list` looks something like this:  
> `[{config`$_{00}$ `: count`$_{00}$ `, config`$_{01}$ ` : count`$_{01}$ `, ...},`  
> &nbsp;&nbsp;`{config`$_{10}$ `: count`$_{10}$ `, config`$_{11}$ ` : count`$_{11}$ `, ...}, ...]`

In [10]:
# The SFS at the 0th locus
print sfs_list[0]

{((2, 12), (10, 0)): 24, ((14, 0), (0, 10)): 122, ((2, 12), (0, 10)): 11, ((5, 9), (0, 10)): 9, ((9, 5), (5, 5)): 2, ((8, 6), (10, 0)): 42, ((12, 2), (8, 2)): 1, ((12, 2), (9, 1)): 3, ((10, 4), (0, 10)): 4, ((0, 14), (5, 5)): 4, ((7, 7), (3, 7)): 1, ((7, 7), (0, 10)): 6, ((0, 14), (7, 3)): 8, ((7, 7), (6, 4)): 2, ((5, 9), (9, 1)): 1, ((6, 8), (2, 8)): 1, ((7, 7), (1, 9)): 1, ((0, 14), (1, 9)): 51, ((13, 1), (2, 8)): 2, ((3, 11), (1, 9)): 1, ((11, 3), (0, 10)): 4, ((9, 5), (0, 10)): 5, ((11, 3), (1, 9)): 4, ((6, 8), (0, 10)): 6, ((10, 4), (2, 8)): 1, ((2, 12), (1, 9)): 1, ((7, 7), (7, 3)): 1, ((7, 7), (10, 0)): 55, ((9, 5), (10, 0)): 83, ((4, 10), (5, 5)): 1, ((5, 9), (10, 0)): 27, ((6, 8), (7, 3)): 1, ((1, 13), (3, 7)): 1, ((4, 10), (8, 2)): 1, ((14, 0), (3, 7)): 33, ((12, 2), (10, 0)): 270, ((11, 3), (10, 0)): 149, ((8, 6), (3, 7)): 1, ((4, 10), (6, 4)): 1, ((4, 10), (9, 1)): 1, ((4, 10), (10, 0)): 27, ((14, 0), (8, 2)): 226, ((14, 0), (6, 4)): 75, ((7, 7), (2, 8)): 2, ((5, 9), (6, 4)

In [11]:
# The combined SFS, across all loci
combined_sfs = momi.sum_sfs_list(sfs_list)

`combined_sfs` looks something like this:
> `{config`$_0$ `: count`$_0$ `, config`$_1$ `: count`$_1$ `, ...}`

Its entries can be accessed by  
```
combined_sfs[config]
```

In [12]:
print "Total number of sites with configuration ((13,1),(10,0),):"
print combined_sfs[((13,1),(10,0),)]

Total number of sites with configuration ((13,1),(10,0),):
9251


# Composite likelihood

The composite likelihood treats each site as independent.  
`momi` provides 2 composite likelihoods:
* **multinomial**: Fixed number of sites, each drawn from multinomial
* **Poisson random field**: Random number of sites, drawn from Poisson distribution

Here we illustrate the **multinomial** likelihood (the default). See **help(momi.composite_log_likelihood)** for more options.

In [13]:
print "Composite log likelihood:", momi.composite_log_likelihood(combined_sfs, demo)

Composite log likelihood: -6855.05740059


# Automatic differentiation

`momi` uses the package `autograd` to automatically compute derivatives.

Gradients can be extremely useful in parameter inference.
However, computing gradients is **not** necessary for much functionality.
Users who don't plan to use auto-differentiation can skip this section, or return to it later.

Let's start by defining a function that maps some parameters to a demography.

In [14]:
import autograd.numpy as np ## thinly wrapped version of numpy for auto-differentiation

def demo_func(N_chb_bottom, N_chb_top, pulse_t, pulse_p, ej_chb, ej_yri):
    ej_chb = pulse_t + ej_chb
    ej_yri = ej_chb + ej_yri
    
    # use autograd.numpy for math functions (e.g. logarithm)
    # This will allow us to take derivatives later
    G_chb = -np.log(N_chb_top / N_chb_bottom) / ej_chb
    
    events = [('-en', 0., 'chb', N_chb_bottom),
              ('-eg', 0, 'chb' , G_chb),
              ('-ep', pulse_t, 'chb', 'nea', pulse_p),
              ('-ej', ej_chb, 'chb', 'yri'),
              ('-ej', ej_yri, 'yri', 'nea'),
              ]

    return momi.Demography(events, ('yri','chb'), (14,10))

Next, let's define a function that returns the expected
TMRCA from the parameters, and then use `autograd` to compute its gradient.

In [15]:
# function mapping vector of parameters to the TMRCA of the corresponding demography
def tmrca_func(params):
    # equivalent to demo_func(params[0], params[1], params[2], ...)
    demo = demo_func(*params)
    # return expected TMRCA
    return momi.expected_tmrca(demo)

# use autograd.grad() to obtain the gradient function
from autograd import grad
tmrca_grad = grad(tmrca_func)

x = [10., .1, .25, .03, .25, 1.]
print "Parameters:"
print x
print "Expected TMRCA:"
print tmrca_func(x)
print "Gradient:"
print tmrca_grad(x)

Parameters:
[10.0, 0.1, 0.25, 0.03, 0.25, 1.0]
Expected TMRCA:
1.31277716218
Gradient:
[0.0045248167517458575, 0.55838496083537514, 0.28449869706692232, 3.9671067152725925, 0.83175299796270241, 0.13916390098743422]


### More details about using `autograd` for automatic differentiation

`autograd` uses the magic of *operator overloading* to compute derivatives automatically.

Here are a few rules to keep in mind, to make sure `autograd` works correctly:

* Arithmetic operations `+,-,*,/,**` all work with autograd

* For more complicated mathematical operations, use `autograd.numpy` and `autograd.scipy`, thinly wrapped versions of `numpy` and `scipy` that support auto-differentiation.
    * For most users, `numpy` contains all the mathematical operations that are needed: `exp()`, `log()`, trigonemetric functions, matrix operations, fourier transform, etc.
    * If needed, it is also possible to use autograd to define derivatives of your own mathematical operations.
* Other do's and don'ts: (copy and pasted from autograd tutorial)
    * Do use
        * Most of `numpy`'s functions
        * Most `numpy.ndarray` methods
        * Some `scipy` functions
        * Indexing and slicing of arrays `x = A[3, :, 2:4]`
        * Explicit array creation from lists `A = np.array([x, y])`
    * Don't use
        * Assignment to arrays `A[0,0] = x`
        * Implicit casting of lists to arrays `A = numpy.sum([x, y])`, use `A = numpy.sum(np.array([x, y]))` instead.
        * `A.dot(B)` notation (use `np.dot(A, B)` instead)
        * In-place operations (such as `a += b`, use `a = a + b` instead)

Documentation for autograd can be found at https://github.com/HIPS/autograd/

# Inference

Now let's try to infer the following demography from data.

In [16]:
true_params = [10., .1, .25, .03, .25, 1.]
true_demo = demo_func(*true_params)

We can generate a new dataset with `ms` (or similar program, e.g. `scrm`, `macs`, `msprime`, etc.)

In [17]:
## to generate new dataset, change this to
## ms_path = "/path/to/ms"
ms_path = None
#ms_path = os.environ["MS_PATH"]

if ms_path is not None:
    print "Generating new dataset with ms"
    
    n_loci = 20
    kb_per_locus = 500
    mut_rate_per_kb, recom_rate_per_kb = 1.,1.
    
    ms_output = momi.simulate_ms(ms_path, true_demo, 
                                 n_loci, kb_per_locus * mut_rate_per_kb, 
                                 additional_ms_params="-r %f %d" % (kb_per_locus*recom_rate_per_kb,
                                                                    int(kb_per_locus * 1000)))
    seg_sites = momi.seg_sites_from_ms(ms_output, sampled_pops=true_demo.sampled_pops)

    sfs_list = momi.get_sfs_list(seg_sites)
    combined_sfs = momi.sum_sfs_list(sfs_list)
    
    ## uncomment these lines to save the new dataset
    #with open(data_filename,'w') as f:
    #    momi.write_seg_sites(f, seg_sites)
else:
    print "No ms_path provided, using SFS previously stored in %s." % data_filename

No ms_path provided, using SFS previously stored in tutorial_data.txt.


In [18]:
# define (lower,upper) bounds on the parameter space
bounds = [(.01, 100.),
          (.01, 100.),
          (.01,5.),
          (.001,.25),
          (.01,5.),
          (.01,5.)]

# pick a random start value for the parameter search
import random
lower_bounds, upper_bounds = [l for l,u in bounds], [u for l,u in bounds]
start_params = [random.triangular(lower, upper, mode)
                for lower, upper, mode in zip(lower_bounds, upper_bounds, [1, 1, 1, .1, 1,1,1])]

Now search for the MCLE (max composite likelihood estimate) with **`momi.composite_mle_search()`**.

By default, `momi.composite_mle_search()` assumes `demo_func` is differentiable with `autograd`, and uses the gradient in a hill-climbing algorithm.
* If you don't want to use `autograd`, you can disable the gradient (i.e. jacobian) with:
    * `momi.composite_mle_search(..., jac=False, ...)`
* Conversely, `momi.composite_mle_search()` provides options for using the Hessian (second-order derivative), in addition to the gradient.
    * See **`help(momi.composite_mle_search)`**.

Beware that **`momi.composite_mle_search()`** can be vulnerable to local maxima.
It isn't a problem in this example, but in other cases, you may have to try multiple starting positions to reach the global maximum.

In [19]:
mcle_search_result = momi.composite_mle_search(combined_sfs, demo_func, start_params, 
                                              bounds = bounds, maxiter = 500, output_progress = 25)

iter 0
objective ( [ 7.57941804  0.77204882  3.12772809  0.12996554  2.94179962  1.28238208] ) == 74041.4
iter 25
objective ( [  3.08230381e+01   1.00000000e-02   1.00000000e-02   1.25180626e-01   4.45735083e-01   2.43734775e-01] ) == 4532.87
iter 50
objective ( [ 6.12955022  0.06138432  0.01        0.15064413  0.54413005  0.08099848] ) == 3128.67
iter 75
objective ( [ 18.60179884   0.02346068   0.27496664   0.13726652   0.2281686    0.36147517] ) == 775.37
iter 100
objective ( [ 13.07266173   0.07073185   0.27770913   0.06411688   0.21931314   0.62521405] ) == 636.163
iter 125
objective ( [ 10.46994694   0.09766749   0.25851679   0.03568675   0.25452852   0.91000949] ) == 609.11
iter 150
objective ( [ 10.43306421   0.09825792   0.25819617   0.03518571   0.25504955   0.91959272] ) == 609.096


In [20]:
print "Search results:"
# print info such as whether search succeeded, number function/gradient evaluations, etc
print mcle_search_result
# note the printed function & gradient values are for -1*log_likelihood

Search results:
  status: 1
 success: True
    nfev: 164
     fun: 609.09631110844202
       x: array([ 10.43286544,   0.09826013,   0.2581945 ,   0.03518365,
         0.25505259,   0.91963437])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
     jac: array([ 0.00019125,  0.02477783,  0.01185449,  0.08726422,  0.0219017 ,
        0.00148004])
     nit: 35


In [21]:
est_params = mcle_search_result.x
print "Estimated params:"
print est_params
print "Ratio of Estimated/Truth:"
print est_params / true_params

Estimated params:
[ 10.43286544   0.09826013   0.2581945    0.03518365   0.25505259
   0.91963437]
Ratio of Estimated/Truth:
[ 1.04328654  0.98260133  1.03277799  1.17278822  1.02021036  0.91963437]


In [22]:
print "Log-likelihood at estimated parameters:", -mcle_search_result.fun
print "Log-likelihood at true parameters:", momi.composite_log_likelihood(combined_sfs, true_demo)

Log-likelihood at estimated parameters: -609.096311108
Log-likelihood at true parameters: -620.223208703


# Confidence Intervals

As the number of independent loci goes to infinity,
the MCLE is asymptotically Gaussian, with mean at the truth,
and covariance given by the inverse 'Godambe information'.

This can be used to construct approximate confidence intervals,
which have the correct coverage properties in the limit (assuming certain regularity conditions, e.g. *identifiability* and *consistency*).

`momi` currently has 2 methods for computing confidence intervals:
* **iid**: Treats the loci as iid. 
    * Appropriate when there are many loci, roughly identically distributed.
* **series**: Treat the segregating sites as a time series. 
    * Works for a small number loci, and they don't have to be identically distributed.
    * Converges as number of SNPs per locus $\to \infty$

In [23]:
print "Computing approximate covariance of MCLE..."

## the approximate covariance matrix of the MCLE
mcle_cov = momi.godambe_scaled_inv("series", est_params, seg_sites, demo_func)
print mcle_cov

Computing approximate covariance of MCLE...
[[  2.83623120e-01  -2.22496489e-03   1.25416135e-03   9.62915545e-04
   -2.99547984e-03  -1.35531531e-02]
 [ -2.22496489e-03   4.74062846e-05   2.23711712e-06  -2.87674454e-05
    2.20243318e-05   5.43452250e-04]
 [  1.25416135e-03   2.23711712e-06   1.30071384e-04   8.36556947e-06
   -1.06969840e-04   4.28419893e-04]
 [  9.62915545e-04  -2.87674454e-05   8.36556947e-06   3.88465261e-05
   -1.97495089e-05  -6.14224853e-04]
 [ -2.99547984e-03   2.20243318e-05  -1.06969840e-04  -1.97495089e-05
    1.69210714e-04  -1.55558427e-04]
 [ -1.35531531e-02   5.43452250e-04   4.28419893e-04  -6.14224853e-04
   -1.55558427e-04   1.61664647e-02]]


In [24]:
# marginal confidence intervals (wald test)
print "Approximate 95% confidence intervals for parameters:"

import scipy.stats
conf_lower, conf_upper = scipy.stats.norm.interval(.95, loc = est_params, scale = np.sqrt(np.diag(mcle_cov)))
print pandas.DataFrame({"Truth" : true_params, "Lower" : conf_lower, "Upper" : conf_upper}, columns = ["Lower","Upper","Truth"])

Approximate 95% confidence intervals for parameters:
      Lower      Upper  Truth
0  9.389062  11.476669  10.00
1  0.084765   0.111755   0.10
2  0.235841   0.280548   0.25
3  0.022968   0.047400   0.03
4  0.229557   0.280548   0.25
5  0.670430   1.168839   1.00


In [25]:
# wald test: residual * cov^{-1} * residual should be Chi-squared with n_params degrees of freedom
print "Approximate p-value of the residuals"

inv_cov = np.linalg.inv(mcle_cov)
# make sure the numerical inverse is still symmetric
assert np.allclose(inv_cov, inv_cov.T)
inv_cov = (inv_cov + inv_cov.T) / 2.0

resids = est_params - true_params
wald_stat = np.dot(resids, np.dot(inv_cov, resids))
print "p = ", 1.-scipy.stats.chi2.cdf(wald_stat, df=len(resids))

Approximate p-value of the residuals
p =  0.513720697517


In [26]:
## log likelihood ratio test
## has a nonstandard null distribution for composite likelihood, which we simulate from
n_null_sims = 10000
llr_p = momi.log_lik_ratio_p("series", n_null_sims, est_params, true_params, [True] * len(true_params), 
                             seg_sites, demo_func)
print "p-value of log-likelihood ratio"
print llr_p

p-value of log-likelihood ratio
0.6172


In [27]:
## log likelihood for iid
n_null_sims = 10000
llr_p = momi.log_lik_ratio_p("iid", n_null_sims, est_params, true_params, [True] * len(true_params), 
                             seg_sites, demo_func)
print "p-value of log-likelihood ratio"
print llr_p

p-value of log-likelihood ratio
0.8376
